In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
# Initialize WebDriver
driver = webdriver.Chrome()
url = "https://www.bseindia.com/markets/equity/EQReports/bulk_deals.aspx"
driver.get(url)

In [3]:
def fetch_bulk_deals():
    """Scrape full bulk deals table from BSE and save to Excel"""
    try:
        # Wait for the table to load completely
        WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='ContentPlaceHolder1_gvbulk_deals']"))
        )

        # Scroll dynamically until no new data appears
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Allow time for new rows to appear
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:  # Stop if no new rows appear
                break
            last_height = new_height

        # Find the table
        table = driver.find_element(By.XPATH, "//*[@id='ContentPlaceHolder1_gvbulk_deals']")
        
        # Find headers
        headers = [th.text.strip() for th in table.find_elements(By.XPATH, "./tbody/tr[1]/th")]

        # Find all rows dynamically
        rows = table.find_elements(By.XPATH, "./tbody/tr[position()>1]")
        print(f"✅ Extracting {len(rows)} rows...")

        # Extract table data
        data = []
        for row in rows:
            cols = [col.text.strip() for col in row.find_elements(By.TAG_NAME, "td")]
            if cols:
                data.append(cols)

        # Convert to DataFrame
        df = pd.DataFrame(data, columns=headers)

        # Save to Excel file
        excel_file_path = r"C:\Users\HP\Downloads\bse_bulk_deals.xlsx"
        df.to_excel(excel_file_path, index=False, engine="openpyxl")

        print(f"✅ Data scraped and saved to '{excel_file_path}' ({len(df)} rows)")

        return df

    except Exception as e:
        print("⚠️ Error fetching table:", e)
        return pd.DataFrame()  # Return empty DataFrame if error

# Run the function
df = fetch_bulk_deals()

# Close the browser
driver.quit()


✅ Extracting 62 rows...
✅ Data scraped and saved to 'C:\Users\HP\Downloads\bse_bulk_deals.xlsx' (62 rows)


In [4]:
df = pd.read_csv('bse_bulk_deals.csv') 
df.head()

,Deal Date,Security Code,Security Name,Client Name,Deal Type *,Quantity,Price **
0,12/02/2025,544224,AFCOM,VICTUS ENTERPRISE LLP,S,"141,840",728.04
1,12/02/2025,544224,AFCOM,VICTUS ENTERPRISE LLP,B,"6,240",694.21
2,12/02/2025,544353,AMWILL,MULTIPLIER SHARE & STOCK ADVISORS PRIVATE LIMITED,B,"105,600",84.49
3,12/02/2025,544353,AMWILL,EPITOME TRADING AND INVESTMENTS,B,"150,000",88.85
4,12/02/2025,544353,AMWILL,VIKASA INDIA EIF I FUND - SHARE CLASS P,S,"230,400",84.45
